# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
data = spark.read.csv('./data/movielens_ratings.csv',inferSchema = True, header = True)

In [7]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [8]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [10]:
# Dividr el data set en training y test
(training, test) = data.randomSplit([0.8, 0.2])

In [13]:
# Construir el modelo de recomendación usando ASL en los datos de entrenamiento
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [14]:
# Entrenar el model
model = als.fit(training)

In [15]:
# Evaluar el modelo con los datos de test
predictions = model.transform(test)

In [16]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   3.0|     8|  1.6480157|
|     85|   3.0|     1|  1.7171869|
|     85|   1.0|    15|-0.98350435|
|     65|   1.0|    28|  3.5103767|
|     65|   1.0|     2|  0.9398716|
|     53|   1.0|    12| -1.4478774|
|     53|   1.0|     7|  1.8688562|
|     53|   5.0|    21|  3.7888327|
|     78|   1.0|    13|   0.545064|
|     78|   1.0|    20|  1.2815145|
|     78|   1.0|     4| 0.52694136|
|     34|   1.0|    15|  2.3029313|
|     81|   1.0|    19| 0.09795587|
|     81|   3.0|    18|  1.8543555|
|     76|   1.0|     2|  2.1496065|
|     26|   1.0|     7|  1.1801645|
|     26|   1.0|    18|-0.54253584|
|     27|   1.0|    28|  1.4033445|
|     27|   1.0|    26|   1.833897|
|     27|   5.0|    23|   1.526382|
+-------+------+------+-----------+
only showing top 20 rows



In [17]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.6093002407536507


El RMSE es mayor que 1, significa que los errores promedio entre las predicciones y los valores reales son relativamente grandes. 